In [1]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 591.8MB 31kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x60a6e000 @  0x7f82414532a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 21.6MB/s 
    100% |████████████████████████████████| 2.0MB 3.9MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
from torch import cuda,nn,optim,utils
from torch.nn import functional as F
from torch.autograd import Variable
from torchvision import datasets,transforms

#DCGAN Model (Radford et.al 2015)

In [0]:
#ThePolice

class discriminator(nn.Module):
    # initializers
    def __init__(self, d):
        super(discriminator, self).__init__()
        self.conv1 = nn.Conv2d(1, d, 4, 2, 1)
        self.conv2 = nn.Conv2d(d, d*2, 4, 2, 1)
        self.conv2_bn = nn.BatchNorm2d(d*2)
        self.conv3 = nn.Conv2d(d*2, d*4, 4, 2, 1)
        self.conv3_bn = nn.BatchNorm2d(d*4)
        self.conv4 = nn.Conv2d(d*4, d*8, 4, 2, 1)
        self.conv4_bn = nn.BatchNorm2d(d*8)
        self.conv5 = nn.Conv2d(d*8, 1, 4, 1, 0)

    # weight_init... for stable gradient flow across layers
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)
    
    # forward method
    def forward(self, input):
        x = F.leaky_relu(self.conv1(input), 0.2)             #LeakyReLU in all layers except last
        x = F.leaky_relu(self.conv2_bn(self.conv2(x)), 0.2)
        x = F.leaky_relu(self.conv3_bn(self.conv3(x)), 0.2)
        x = F.leaky_relu(self.conv4_bn(self.conv4(x)), 0.2)
        x = F.sigmoid(self.conv5(x))

        return x
      
#TheCulprit

class generator(nn.Module):
    # initializers
    def __init__(self, d):
        super(generator, self).__init__()
        self.deconv1 = nn.ConvTranspose2d(100, d*8, 4, 1, 0)     #100-d latent space vector
        self.deconv1_bn = nn.BatchNorm2d(d*8)
        self.deconv2 = nn.ConvTranspose2d(d*8, d*4, 4, 2, 1)
        self.deconv2_bn = nn.BatchNorm2d(d*4)
        self.deconv3 = nn.ConvTranspose2d(d*4, d*2, 4, 2, 1)
        self.deconv3_bn = nn.BatchNorm2d(d*2)
        self.deconv4 = nn.ConvTranspose2d(d*2, d, 4, 2, 1)
        self.deconv4_bn = nn.BatchNorm2d(d)
        self.deconv5 = nn.ConvTranspose2d(d, 1, 4, 2, 1)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
        # x = F.relu(self.deconv1(input))
        x = F.relu(self.deconv1_bn(self.deconv1(input)))  #ReLU all layers except last where tanh used
        x = F.relu(self.deconv2_bn(self.deconv2(x)))
        x = F.relu(self.deconv3_bn(self.deconv3(x)))
        x = F.relu(self.deconv4_bn(self.deconv4(x)))
        x = F.tanh(self.deconv5(x))

        return x
      
#WeightInitialiser
def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

**Utilities**

In [0]:
#As mentioned in the paper
param = {
    "batch_sz" : 128,
    "lr" : 0.0002,
    "beta" : 0.5,
    "tr_epoch" : 15,
    "img_size" : 64}

#data transforms
transform = transforms.Compose([
        transforms.Resize(param["img_size"]),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

#Load MNIST
train_loader = utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transform),
    batch_size=param["batch_sz"], shuffle=True)


In [0]:
#Model_init
g = generator(128)
d = discriminator(128)
g.weight_init(mean=0.0, std=0.02)
d.weight_init(mean=0.0, std=0.02)

#cost function
criterion = nn.BCELoss()

if cuda.is_available() :
  g.cuda() 
  d.cuda()
  criterion.cuda()

In [0]:
g_optim = optim.Adam(g.parameters(), lr=param["lr"], betas=(param["beta"], 0.999))
d_optim = optim.Adam(d.parameters(), lr=param["lr"], betas=(param["beta"], 0.999))